# YOLO-X 기반의 드론 인식 모델 생성

## 0. 사전 환경 설정

### 0.1 WanDB 로깅 설정

In [ ]:
!pip install wandb

<API-Key>에 본인의 WandDB 프로젝트 API키를 입력

In [ ]:
!wandb login <API-Key>

### 0.2 TensorRT 설치

In [ ]:
!sudo apt install tensorrt

## 1. YOLO-X Git 리포지토리 복제

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git

## 2. YOLO-X 사용 환경 구성

In [ ]:
%cd YOLOX
!pip3 install -v -e .

## 3. 인식할 객체 클래스 재정의
`yolox/data/datasets/voc_classes.py` 파일에 정의 되어 있는 20개의 객체 클래스가 아닌 drone 객체 클래스만 정의

In [ ]:
from IPython.core.magic import register_line_cell_magic
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate yolox/data/datasets/voc_classes.py

VOC_CLASSES = (
  "drone",
)

In [ ]:
%%writetemplate yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
  "drone",
)

## 4. 기존 제공되는 YOLOX weights 파일 다운로드

[YOLOX GitHub releases](https://github.com/Megvii-BaseDetection/YOLOX/releases) 에서 제공

In [ ]:
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_nano.pth

In [ ]:
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_tiny.pth

In [ ]:
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth

In [ ]:
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_m.pth

## 5. 라벨링 완료된 drone 데이터셋 다운로드
아래 파일구조로 구성
```
VOCdevkit
  |___VOC2012
     |___Annotations
     |___ImageSets
            |___Main
                   |___train.txt
                   |___valid.txt
     |___JPEGImages
```

In [ ]:
%cd datasets
!wget https://www.dropbox.com/s/9c0jbixl7df7b0w/Drone-dataset.zip?dl=1 -O VOCdevkit.zip -q --show-progress
!unzip -qq VOCdevkit.zip
!rm VOCdevkit.zip
%cd ..

## 6. 학습 패러미터 설정

YOLOX에서 학습 패러미터는 experiment file에 저장 된다. 예시 파일은 `exps/example/yolox_x_voc/` 에 저장된다.

스크립트들은 아래와 같은 패러미터들을 제공한다:

* **네트워크 깊이와 너비**
* **Epoch 수**
* **객체 클래스의 수**
* **Augmentation 관련 수치**
* **학습 및 검증 데이터셋 경로**


자세한 양식은 `yolox/exp/yolox_base.py` 참조.

YOLOX에서 기존에 정의된 데이터셋 경로 관련 규칙: <br><br> `datasets/VOCdevkit/VOC` + `str(year)` + `/ImageSets/Main`.

이에 따라 `get_dataloader` 와 `get_eval_loader` 함수에서 year값에 해당하는 **2007** 혹은 **2012** 만 정의하면 된다.

```
image_sets=[('2012', 'train)],
image_sets=[('2012','valid')],
```
해당 모델은 YOLOX-M을 기반으로 작성되었다.

### 6.1 Experiments 파일 정의

In [3]:
%cd exps
!wget https://github.com/fanntom/2023-capstone-yolox/blob/main/ExpConfigs.zip -O custom_exps.zip -qq --show-progress
!unzip custom_exps.zip
%cd ..

/content/YOLOX/exps
custom_exps.zip         [ <=>                ] 137.91K  --.-KB/s    in 0.02s   
Archive:  custom_exps.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of custom_exps.zip or
        custom_exps.zip.zip, and cannot find custom_exps.zip.ZIP, period.
/content/YOLOX


YOLOX-M을 활용하기 위해 패러미터들을 다음과 같이 설정한다:

* **Depth** : 0.67
* **Width** : 0.75
* **Epochs** : 25
* **Number of Classes** : 1
* **Train Data Path** : `image_sets=[('2012', 'train)],`
* **Validation Data Path** : `image_sets=[('2012','valid')],`
* **Augmentation**
    - Mixup: 1.0
    - Mosaic: 1.0
    - HSV: 1.0
    - FLIP: 0.5

Epoch의 수를 50로 수정(기존 Value는 300)

In [ ]:
MAX_EPOCH = 50
!sed -i -e 's/self.max_epoch = 300/self.max_epoch = {MAX_EPOCH}/g' "exps/ExpConfigs/yolox_voc_m.py"

## 7. 학습진행

아래 코드는 WanDB Logging 사용시 활용

In [ ]:
!python tools/train.py -f exps/ExpConfigs/yolox_voc_m.py -d 1 -b 16 --fp16 -o -c yolox_m.pth --logger wandb \
                wandb-project <WanDB-Project-Name>

아래 코드는 WanDB 로깅 미사용시 활용

In [ ]:
!python tools/train.py -f exps/ExpConfigs/yolox_voc_m.py -d 1 -b 16 --fp16 -o -c yolox_m.pth

## 8. 모델 검증

In [ ]:
MODEL_PATH = "YOLOX_outputs/yolox_voc_m/best_ckpt.pth"
!python3 tools/eval.py -c {MODEL_PATH} -b 16 -d 1 --conf 0.001 -f exps/ExpConfigs/yolox_voc_m.py

## 9. 이미지를 사용한 학습된 모델 사용


In [ ]:
# Download Images
%mkdir inference_media
%cd inference_media
!wget https://www.dropbox.com/s/1dy29ys1fkce8k3/bird-and-drone.png?dl=1 -O bird-and-drone.jpg -qq --show-progress
!wget https://www.dropbox.com/s/i0afm1nqm6iiuji/eagle-capturing-drone.png?dl=1 -O eagle-capturing-drone.jpg -qq --show-progress
!wget https://www.dropbox.com/s/kje4h0avj2scgjj/eagle-vs-drone.png?dl=1 -O eagle-vs-drone.jpg -qq --show-progress
!wget https://www.dropbox.com/s/jhjy3lfl5908vta/drone-vs-birds.jpg?dl=1 -O drone-vs-birds.jpg -qq --show-progress
%cd ..

In [ ]:
!python tools/demo.py image -f exps/ExpConfigs/yolox_voc_m.py -c {MODEL_PATH} --path ./inference_media/ --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

In [ ]:
import matplotlib.pyplot as plt
import cv2
import glob

In [ ]:
images = []
for file in glob.glob("YOLOX_outputs/yolox_voc_m/vis_res/**/*.jpg"):
  img = cv2.imread(file)
  images.append(img)

In [ ]:
plt.figure(figsize=(25,15))
for i in range(len(images)):
  plt.subplot(1, len(images), i+1);
  plt.imshow(images[i][...,::-1]);

## 10. 동영상을 사용한 학습된 모델 사용

In [ ]:
!wget https://www.dropbox.com/s/u1kqu0yxj07e35e/Drones-1-original.mp4?dl=1 -O Drones-1-original.mp4 -qq --show-progress

In [ ]:
!python tools/demo.py video -f exps/ExpConfigs/yolox_voc_m.py -c {MODEL_PATH} --path Drones-1-original.mp4 --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

## 11. 모델 양자화 과정

Work In Progress

In [ ]:
!pip install openvino

In [ ]:
!pip install onnxruntime

In [ ]:
!python tools/export_onnx.py --output-name yolox-before-quanta.onnx -f exps/ExpConfigs/yolox_voc_m.py -c YOLOX_outputs/yolox_voc_m/best_ckpt.pth

In [ ]:
!pip install openvino-dev

In [ ]:
!pip install openvino-dev[onnx]

In [ ]:
!pip install openvino-dev[torch]

In [ ]:
!mo --input_model yolox-before-quanta.onnx

In [ ]:
!pot -q default -m yolox-before-quanta.xml -w yolox-before-quanta.bin --engine simplified --data-source datasets/VOCdevkit/VOC2012/JPEGImages --output-dir .